In [1]:
import numpy as np
import pandas as pd
from random import shuffle

In [2]:
prob=pd.read_csv('letters_prob.csv', index_col='letter') # preprocessed file
decipher_form=pd.read_csv('cipher.csv', index_col='Unnamed: 0') # preprocessed file
transition=pd.read_csv('letter_transition_matrix.csv', index_col='letter')
alphabet=list('abcdefghijklmnopqrstuvwxyz .')
trans_array=np.array(transition)
alp_num=dict(zip(alphabet, list(range(len(alphabet))))) # We need it when convert text to numbers

In [3]:
# prepare text
text=open('ciphertext.txt', 'r').read()
text=text.replace('\n', ' ')
text=text.replace('  ', ' ')
text=list(text)

In [4]:
def decipher(y,f):
    dictionary=dict(zip(f, alphabet))
    x=[]
    for let in y:
        x.append(dictionary[let])
    
    return x

def tonum(y): # convert text to numbers
    x=[]
    for let in y:
        x.append(alp_num[let])
    
    return x
    

def LogPyf(y,f):
    y=decipher(y,f)
 
    Ps=np.zeros(len(y))
    
    Ps[0]=np.log(prob.loc[y[0]]['Unnamed: 0'])
    y=tonum(y)
    Ps[1:]=np.log(trans_array[y[1:],y[:len(y)-1]])

    P=Ps.sum()
    return P

In [5]:
LogPyf(text, decipher_form.index)

-64363.198159222811

In [110]:
# Initialize f_n to avoid -infs
while True:
    alph=list(alphabet)
    shuffle(alph)
    f_n=np.array(alph)
    if LogPyf(text,f_n)!= -np.inf:
        break

/home/daniel/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log


In [111]:
leave_or_not=np.array([True,False])
normalization_factor=np.log(len(text))*10

f=[]
Prob=[]
f.append(str(f_n))
Prob.append(LogPyf(text,f_n))
N=len(f_n)

for i in range(5000):
    re=np.random.choice(N, 2, replace=False)
    f_new=np.array(f_n)
    f_new[re]=f_new[np.flip(re,0)]
    
    crit=LogPyf(text, f_new)-LogPyf(text, f_n)
    
    if crit>=0:
        f.append(str(f_new))
        f_n=f_new
        Prob.append(LogPyf(text,f_new))
    
    else:
        alpha=np.exp(crit/normalization_factor)
        accept=np.random.choice(leave_or_not,p=[alpha,1-alpha])
        
        if accept:
            f.append(str(f_new))
            f_n=f_new
            Prob.append(LogPyf(text,f_new))
    

/home/daniel/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log


In [112]:
max(set(f), key=f.count) # maximum repeating string in the list of strings 'f'

"['g' 's' 'c' 'p' 'b' 'e' 'd' 'i' 'o' 'w' 'k' 'n' 'z' 'r' '.' 'u' 'm' 'a'\n ' ' 'x' 'h' 'q' 'f' 'v' 'l' 't' 'y' 'j']"

In [98]:
np.array(decipher_form.index)

array(['g', 's', 'c', 'p', 'b', 'e', 'd', 'i', 'o', 'w', 'k', 'n', 'z',
       'r', '.', 'u', 'm', 'a', ' ', 'x', 'h', 't', 'f', 'v', 'l', 'q',
       'y', 'j'], dtype=object)